In [19]:
from compas.datastructures import Mesh
import compas.datastructures.mesh.subdivision as sd
from compas.geometry import Circle
import ipyvolume as ipv
from utilities import draw_compas_mesh
from shapes import *
from compas.datastructures import meshes_join
from compas.geometry import distance_point_point


origin = (0,0,0)
normal = (0,0,1)
plane = (origin, normal)
r_axis = 5
r_pipe = .5

torus = Torus(plane, r_axis, r_pipe)

vs, fs = torus.to_vertices_and_faces(u = 12, v = 4)

mesh = Mesh.from_vertices_and_faces(vs, fs)

#subd = sd.mesh_subdivide_catmullclark(mesh, k=1)

#draw_compas_mesh(subd)

draw_compas_mesh(mesh)
#mesh = subd

Mesh(color=array('white', dtype='<U5'), line_material=ShaderMaterial(), material=ShaderMaterial(side='DoubleSi…

In [33]:
distances = []

for v in mesh.vertices():
    v_coord = mesh.vertex_coordinates(v)
    distance_to_mid = distance_point_point(origin, v_coord)
    distances.append(distance_to_mid)

print(distances)

[5.499999999999999, 5.024937810560445, 4.5, 5.024937810560445, 5.5, 5.024937810560444, 4.499999999999999, 5.024937810560444, 5.5, 5.024937810560444, 4.499999999999999, 5.024937810560444, 5.499999999999999, 5.024937810560445, 4.5, 5.024937810560445, 5.499999999999999, 5.024937810560444, 4.499999999999999, 5.024937810560444, 5.499999999999999, 5.024937810560446, 4.5, 5.024937810560446, 5.499999999999999, 5.024937810560445, 4.5, 5.024937810560445, 5.499999999999999, 5.024937810560446, 4.5, 5.024937810560446, 5.5, 5.024937810560446, 4.5, 5.024937810560446, 5.499999999999999, 5.024937810560445, 4.5, 5.024937810560445, 5.5, 5.024937810560445, 4.499999999999999, 5.024937810560445, 5.5, 5.024937810560444, 4.5, 5.024937810560444]


In [38]:
distance_outer_ring = round(max(distances), 1)
distance_inner_ring = round(min(distances), 1)

print(distance_inner_ring)
print(distance_outer_ring)

4.5
5.5


In [45]:
point_list = []

for v in mesh.vertices():
    v_coord = mesh.vertex_coordinates(v)
    point_list.append(v_coord)
    distance_to_mid = distance_point_point(origin, v_coord)
     
    if distance_to_mid == distance_outer_ring:
        mesh.set_vertex_attribute(v, 'location', 'outer')
    else:
        mesh.set_vertex_attribute(v, 'location', 'inner')

        
new_mesh = Mesh()

for pt in point_list:
    new_mesh.add_vertex(x=pt[0], y=pt[1], z=pt[0])
        
print(point_list)
new_mesh = Mesh.from_points(point_list)
draw_compas_mesh(new_mesh)



[[3.889087296526011, 3.889087296526011, 0.0], [3.5355339059327373, 3.5355339059327373, 0.4999999999999999], [3.181980515339464, 3.181980515339464, 6.123233995736765e-17], [3.5355339059327373, 3.5355339059327373, -0.4999999999999999], [1.4235047480638647, 5.312592044589875, 0.0], [1.2940952255126041, 4.829629131445341, 0.4999999999999999], [1.1646857029613438, 4.346666218300807, 6.123233995736765e-17], [1.2940952255126041, 4.829629131445341, -0.4999999999999999], [-1.4235047480638636, 5.312592044589875, 0.0], [-1.2940952255126033, 4.829629131445341, 0.4999999999999999], [-1.1646857029613427, 4.346666218300807, 6.123233995736765e-17], [-1.2940952255126033, 4.829629131445341, -0.4999999999999999], [-3.8890872965260104, 3.8890872965260113, 0.0], [-3.5355339059327373, 3.5355339059327373, 0.4999999999999999], [-3.181980515339464, 3.181980515339464, 6.123233995736765e-17], [-3.5355339059327373, 3.5355339059327373, -0.4999999999999999], [-5.312592044589874, 1.4235047480638654, 0.0], [-4.829629

TypeError: 'bool' object is not iterable